In [1]:
import numpy as np
from pyMuellerMat.common_mm_functions import diattenuator_retarder_function2
from pyMuellerMat import MuellerMat

# === USER SETTINGS ===
wavelength_nm = 675               # Incident wavelength in nm
aoi_deg = 10                      # Angle of incidence in degrees
n_complex = 1.73 + 1j * 7.94      # Aluminum index of refraction at 675 nm
# ======================

# Convert angle to radians
aoi_rad = np.radians(aoi_deg)
n1 = 1.0  # Ambient medium (air)

# === Fresnel coefficients for metallic mirror ===
theta_i = aoi_rad
theta_t = np.arcsin(n1 * np.sin(theta_i) / n_complex)  # Snell's Law (complex)

# Components
cos_theta_i = np.cos(theta_i)
cos_theta_t = np.sqrt(1 - (n1 * np.sin(theta_i) / n_complex)**2)

# Fresnel reflection coefficients (complex)
rs = ((n1 * cos_theta_i - n_complex * cos_theta_t) /
      (n1 * cos_theta_i + n_complex * cos_theta_t))
rp = ((n_complex * cos_theta_i - n1 * cos_theta_t) /
      (n_complex * cos_theta_i + n1 * cos_theta_t))

# Reflectance (real)
R_s = np.abs(rs)**2
R_p = np.abs(rp)**2

# print("Rs: " + str(R_s))
# print("Rp: " + str(R_p))

# Phase shift between r_p and r_s
delta = np.angle(rp) - np.angle(rs)

# TODO: Check why this is different
# Using values from refractiveindex.info
R_s = 0.90309
R_p = 0.90019
delta = np.radians(13.936 - (-166.486))  # from φp − φs

# === Build Mueller matrix ===
mueller_array = diattenuator_retarder_function2(r1=R_s, r2=R_p, delta=delta)
M1 = MuellerMat.MuellerMatrix(lambda: mueller_array)

# Two mirrors
M2 = MuellerMat.MuellerMatrix(lambda: mueller_array @ mueller_array)

# Evaluate matrices
M1_eval = M1.evaluate()
M2_eval = M2.evaluate()

# Optional manual retardance, diattenuation, and rotation angle extraction
def compute_diattenuation(M):
    return np.abs(M[0, 1]) / M[0, 0]

def compute_retardance(M):
    m22 = M[2:4, 2:4]
    trace = np.trace(m22)
    return np.real(np.arccos(trace / 2))

def compute_rotation_angle(M):
    return np.degrees(np.arctan2(M[1, 0], M[0, 0]))

ret_1, dia_1, rot_1 = compute_retardance(M1_eval), compute_diattenuation(M1_eval), compute_rotation_angle(M1_eval)
ret_2, dia_2, rot_2 = compute_retardance(M2_eval), compute_diattenuation(M2_eval), compute_rotation_angle(M2_eval)

# Display results
print("=== SINGLE MIRROR ===")
print(f"Retardance:     {ret_1 / (2 * np.pi):.4f} waves")
print(f"Diattenuation:  {dia_1:.6f}")
print(f"Rotation Angle: {rot_1:.2f} degrees")
print("Mueller Matrix:\n", M1_eval)

print("\n=== DOUBLE MIRROR ===")
print(f"Retardance:     {ret_2 / (2 * np.pi):.4f} waves")
print(f"Diattenuation:  {dia_2:.6f}")
print(f"Rotation Angle: {rot_2:.2f} degrees")
print("Mueller Matrix:\n", M2_eval)


=== SINGLE MIRROR ===
Retardance:     0.4288 waves
Diattenuation:  0.001608
Rotation Angle: 0.09 degrees
Mueller Matrix:
 [[ 0.90164     0.00145     0.          0.        ]
 [ 0.00145     0.90164     0.          0.        ]
 [ 0.          0.         -0.90161438  0.00664077]
 [ 0.          0.         -0.00664077 -0.90161438]]

=== DOUBLE MIRROR ===
Retardance:     0.0990 waves
Diattenuation:  0.003216
Rotation Angle: 0.18 degrees
Mueller Matrix:
 [[ 0.81295679  0.00261476  0.          0.        ]
 [ 0.00261476  0.81295679  0.          0.        ]
 [ 0.          0.          0.81286439 -0.01197483]
 [ 0.          0.          0.01197483  0.81286439]]
